# Introducción

En este notebook se procesaran los videos y se extraerán las caracteristicas usando el modelo preentrenado SlowFast

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models.video as models
import cv2
import numpy as np
from PIL import Image
from imblearn.over_sampling import SMOTE
import re
import os
import json
import pickle
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import torch
import numpy as np
import cv2
from torchvision import transforms
from fvcore.common.config import CfgNode as CN

In [2]:
#Funciones para agrupar la ruta de los clips por acción y ordenamos el diccionario por el número de la acción

def group_videos_by_action(root_path, video_extensions=None):
    if video_extensions is None:
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']

    grouped_videos = defaultdict(list)

    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                action_name = os.path.basename(dirpath)  # Nombre de la carpeta contenedora
                file_path = os.path.join(dirpath, file)
                grouped_videos[action_name].append(file_path)
    
    sorted_actions = sorted(
        grouped_videos.items(), 
        key=lambda x: int(x[0].split('_')[1])  # Extraer número después de "action_"
    )
    return grouped_videos



def ordenar_diccionario(input_dict): 
    # Ordenamos las claves extrayendo el número
    sorted_keys = sorted(input_dict.keys(), key=lambda x: int(re.search(r'\d+', x).group()))
    
    # Creamos un nuevo defaultdict ordenado
    sorted_dict = defaultdict(list, {key: input_dict[key] for key in sorted_keys})

    return sorted_dict

root_path = "F:/data/mvfouls/augmented_videos_severity"
videos_path = group_videos_by_action(root_path = root_path)
sorted_keys = ordenar_diccionario(videos_path)


In [3]:
#Dividimos las acciones en grupos para facilitar su procesamiento
def save_dict_chunks(input_dict, chunk_size, save_path, format='npy'):

    keys = list(input_dict.keys())
    for i in range(0, len(keys), chunk_size):
        chunk = {key: input_dict[key] for key in keys[i:i + chunk_size]}
        file_name = f"{save_path}_chunk_{i // chunk_size}.{format}"
        
        if format == 'npy':
            np.save(file_name, chunk)
        elif format == 'pkl':
            with open(file_name, 'wb') as f:
                pickle.dump(chunk, f)
        else:
            raise ValueError("Formato no soportado. Usa 'npy' o 'pkl'.")
        print(f"Guardado: {file_name}")



save_dict_chunks(
    sorted_keys,
    chunk_size=500,  #Número de elementos por archivo
    save_path='F:/data/mvfouls/augmented_severity',
    format='npy' 
)


Guardado: F:/data/mvfouls/augmented_severity_chunk_0.npy
Guardado: F:/data/mvfouls/augmented_severity_chunk_1.npy
Guardado: F:/data/mvfouls/augmented_severity_chunk_2.npy


In [1]:
#Cargamos el chunk a procesar
chunk = np.load('F:/data/mvfouls/augmented_severity_chunk_2.npy', allow_pickle=True).item()


NameError: name 'np' is not defined

In [11]:
import torch
import cv2
import numpy as np
from tqdm import tqdm

# Función para cargar el modelo preentrenado SlowFast
def load_slowfast_model():
    model_name = 'slowfast_r50'  # Puedes usar otras variantes como 'slowfast_r101'
    model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
    model.eval()  # Poner el modelo en modo de evaluación
    return model

# Función para extraer frames de un video en el formato adecuado para SlowFast
def video_to_frames(video_path, frame_size=(224, 224), slow_frames=8, fast_frames=32):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error al abrir el video: {video_path}")
    
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
    
    cap.release()
    
    if len(frames) == 0:
        raise ValueError(f"No se pudieron extraer frames del video: {video_path}")
    
    # Asegurar suficientes frames para ambos flujos
    while len(frames) < fast_frames:
        frames.append(frames[-1])
    
    frames = np.array(frames)
    frames = np.transpose(frames, (0, 3, 1, 2))  # [T, C, H, W]
    
    # Preparar entradas para SlowFast
    slow_idx = np.linspace(0, len(frames) - 1, slow_frames).astype(int)  # Submuestreo para el flujo lento
    fast_idx = np.arange(fast_frames)  # Todos los frames para el flujo rápido
    
    slow_frames_data = frames[slow_idx]  # Frames para el flujo lento
    fast_frames_data = frames[fast_idx]  # Frames para el flujo rápido
    
    return slow_frames_data, fast_frames_data

# Función para extraer características del video utilizando el modelo SlowFast
def extract_features(video_path, model):
    slow_frames, fast_frames = video_to_frames(video_path)  # Extraer frames para ambos flujos
    
    slow_frames = torch.tensor(slow_frames).float() / 255.0  # Normalizar
    fast_frames = torch.tensor(fast_frames).float() / 255.0  # Normalizar
    
    # Ajustar dimensiones: [batch_size, num_channels, num_frames, height, width]
    slow_frames = slow_frames.unsqueeze(0).permute(0, 2, 1, 3, 4)  # [1, C, T_slow, H, W]
    fast_frames = fast_frames.unsqueeze(0).permute(0, 2, 1, 3, 4)  # [1, C, T_fast, H, W]
    
    # SlowFast espera una lista con los dos flujos
    inputs = [slow_frames, fast_frames]
    
    with torch.no_grad():
        features = model(inputs)
    
    return features.cpu().numpy()

# Función principal para extraer características de varios videos
def extract_features_from_grouped_videos(grouped_videos, model):
    action_features = {}
    video_paths_all = []  # Lista global de todos los videos a procesar
    
    # Recopilar todos los caminos de los videos en una lista
    for action, video_paths in grouped_videos.items():
        video_paths_all.extend(video_paths)
    
    # Usamos tqdm para crear una barra de progreso global
    for video_path in tqdm(video_paths_all, desc="Procesando videos", unit="video"):
        action = video_path.split("\\")[-2]  # Obtener el nombre de la acción desde la ruta (ajustar si es necesario)
        
        # Crear un lugar para las características de cada video
        if action not in action_features:
            action_features[action] = []
        
        try:
            features = extract_features(video_path, model)
            action_features[action].append(features)
        except Exception as e:
            print(f"Error al procesar el video {video_path}: {e}")
    
    # Promediar las características de cada acción
    for action, clips in action_features.items():
        action_features[action] = np.mean(clips, axis=0)
    
    return action_features

# Cargar el modelo SlowFast
try:
    slowfast_model = load_slowfast_model()
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    slowfast_model = None

# Extraer características de los videos
if slowfast_model is not None:
    try:
        extracted_features = extract_features_from_grouped_videos(chunk, slowfast_model)
    except Exception as e:
        print(f"Error al extraer características de los videos: {e}")
else:
    print("El modelo no se cargó correctamente, no se puede continuar con la extracción de características.")

Using cache found in C:\Users\juanm/.cache\torch\hub\facebookresearch_pytorchvideo_main
Procesando videos: 100%|██████████████████████████████████████████████████████████| 369/369 [06:41<00:00,  1.09s/video]


{'action_3919': array([[ 1.5924238 , -0.62989664, -0.30801937, -0.7429506 , -0.873611  ,
        -0.0283231 , -0.2162701 , -1.5618458 , -0.9198579 , -1.8432976 ,
        -2.0224593 , -1.7014021 , -0.1792395 , -2.1569374 , -0.16933687,
        -0.9811298 , -0.0463524 , -0.9462986 , -0.55336523, -1.1845268 ,
         0.23410201, -1.128073  ,  2.0362024 ,  2.141557  , -1.3536296 ,
         1.6027302 ,  0.10395966, -1.5060374 ,  0.5864382 ,  0.30569345,
         1.2206515 , -0.9309619 , -1.5759925 ,  0.2580737 , -0.16347194,
        -0.5467866 , -1.1541241 , -0.47849527, -1.6104053 , -1.9483666 ,
         4.428599  , -0.10457247,  1.521773  , -1.6258812 , -1.1274958 ,
         0.824099  , -1.2037324 ,  1.4042481 , -0.41801223,  0.95871764,
        -0.49251094,  0.5849826 , -1.5284585 , -1.8244181 , -0.8234236 ,
         1.3543121 , -0.11096575, -1.6673635 , -2.1532812 ,  0.39149043,
         2.1605227 ,  0.64299744,  1.033591  ,  1.0655066 ,  1.3645277 ,
         3.2301273 , -1.5972081 ,  

In [12]:
# Guardar los resultados parciales
with open('augmented_severity_train_2.pkl', 'wb') as f:
    pickle.dump(extracted_features, f)  # Guarda el diccionario de resultados

In [14]:
#Funciones para juntar los resultados parciales en un único archivo

directory_path = "F:/MVFoulRecognition/features/Slowfast/train/severity"
prefix = "augmented_" # Prefijo para buscar los archivos 

files = []

for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if os.path.isfile(full_path) and filename.startswith(prefix):  # Verifica si es un archivo y tiene el prefijo
        files.append(full_path)

print("Archivos encontrados:")
sorted_files = sorted(files, key=lambda x: int(x.split('_train_')[-1].split('.pkl')[0]))

# Imprime la lista ordenada
for file in sorted_files:
    print(file)
    

combined_data = []

# Cargar y combinar los datos
for file in sorted_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)  # Carga el archivo .pkl
        combined_data.append(data)  # Agrega el array a la lista



# Guardar el array combinado en un nuevo archivo .pkl
with open('test_combined.pkl', 'wb') as f:
    pickle.dump(combined_data, f)

print("Arrays combinados guardados en 'train_augmented.pkl'")


Archivos encontrados:
F:/MVFoulRecognition/features/Slowfast/train/severity\augmented_severity_train_0.pkl
F:/MVFoulRecognition/features/Slowfast/train/severity\augmented_severity_train_1.pkl
F:/MVFoulRecognition/features/Slowfast/train/severity\augmented_severity_train_2.pkl
Arrays combinados guardados en 'train_augmented.pkl'


In [15]:
#Agrupamos en un mismo archivo los resultados de la extraccion de videos y videos aumentados


archivo2 = "F:/MVFoulRecognition/features/Slowfast/train/severity/end/train_augmented.pkl"
archivo1 = "F:/MVFoulRecognition/features/Slowfast/train/end/train_combined.pkl"
archivo_combinado = "train_combinados.pkl"

# Cargar los archivos
with open(archivo1, "rb") as f1, open(archivo2, "rb") as f2:
    datos1 = pickle.load(f1)
    datos2 = pickle.load(f2)

if isinstance(datos1, list) and isinstance(datos2, list):
    datos_combinados = datos1 + datos2  # Unir listas
elif isinstance(datos1, dict) and isinstance(datos2, dict):
    datos_combinados = {**datos1, **datos2}  # Unir diccionarios
else:
    raise TypeError("Los archivos tienen estructuras incompatibles")

# Guardar el archivo combinado
with open(archivo_combinado, "wb") as f:
    pickle.dump(datos_combinados, f)

print(f"Archivos combinados y guardados en '{archivo_combinado}'")


Archivos combinados y guardados en 'train_combinados.pkl'


# A partir de aquí termina la parte de extracción de features y comienza el entrenamiento (podemos mirar otro notebook)

In [16]:
with open('F:/MVFoulRecognition/features/Slowfast/train/severity/end/train_combinados.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_train = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_train = dict(sorted(resultado_train.items(), key=lambda x: int(x[0].split('_')[1])))

#print(dic_ordenado_train)
with open('F:/MVFoulRecognition/features/Slowfast/test/test_combined.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_test = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_test = dict(sorted(resultado_test.items(), key=lambda x: int(x[0].split('_')[1])))

In [19]:
def load_action_labels(json_path):
    """
    Carga las etiquetas de acción desde un archivo JSON.

    Args:
        json_path (str): Ruta al archivo JSON.

    Returns:
        dict: Diccionario donde las claves son nombres de acciones (ej. 'action_0')
              y los valores son las etiquetas (ej. 'Challenge', 'Tackling').
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Offence']  # Extraer la etiqueta de 'Action class'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels



In [22]:
dic_ordenado_train.keys()

dict_keys(['action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7', 'action_8', 'action_9', 'action_10', 'action_11', 'action_12', 'action_13', 'action_14', 'action_15', 'action_16', 'action_17', 'action_18', 'action_19', 'action_20', 'action_21', 'action_22', 'action_23', 'action_24', 'action_25', 'action_26', 'action_27', 'action_28', 'action_29', 'action_30', 'action_31', 'action_32', 'action_33', 'action_34', 'action_35', 'action_36', 'action_37', 'action_38', 'action_39', 'action_40', 'action_41', 'action_42', 'action_43', 'action_44', 'action_45', 'action_46', 'action_47', 'action_48', 'action_49', 'action_50', 'action_51', 'action_52', 'action_53', 'action_54', 'action_55', 'action_56', 'action_57', 'action_58', 'action_59', 'action_60', 'action_61', 'action_62', 'action_63', 'action_64', 'action_65', 'action_66', 'action_67', 'action_68', 'action_69', 'action_70', 'action_71', 'action_72', 'action_73', 'action_74', 'action_75', 'action_76'

In [20]:
ls = load_action_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

ls = load_action_labels("F:/data/mvfouls/test/annotations.json")
ls = sorted(ls.items())
sorted_labels_test = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

In [23]:
sorted_labels_train.extend([('action_' + str(i), 'No offence') for i in range(2916, 3945)])

# Verificar algunas entradas
print(sorted_labels_train[-10:])  # Últimos 10 elementos

# Crear un diccionario con claves numéricas del 0 al 3999
diccionario = {i: 0 for i in range(0, 3945)}

# Recorrer las claves en `res`
for r in dic_ordenado_train:
    # Extraer el número de la clave 'action_X'
    action = int(r.split("_")[1])  # Convertimos el número extraído a entero
    
    # Si el número extraído de `action_X` está en `diccionario`, lo marcamos como 1
    if action in diccionario:
        diccionario[action] = 1

# Mostrar el diccionario actualizado
diccionario_filtrado = {k: v for k, v in diccionario.items() if v == 0}
print(diccionario_filtrado)

[('action_3935', 'No offence'), ('action_3936', 'No offence'), ('action_3937', 'No offence'), ('action_3938', 'No offence'), ('action_3939', 'No offence'), ('action_3940', 'No offence'), ('action_3941', 'No offence'), ('action_3942', 'No offence'), ('action_3943', 'No offence'), ('action_3944', 'No offence')]
{3266: 0, 3569: 0}


In [24]:
sorted_labels_train = [item for item in sorted_labels_train if item[0] != 'action_3266' and item[0] != 'action_3569' ]

for item in sorted_labels_train:
    if item[0] == 'action_3569':
        print(item)
        
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in sorted_labels_train if item[1] == 'Offence']

# 2. Seleccionar las primeras mil acciones con etiqueta 'Offence'
# Aseguramos que hay al menos mil acciones con etiqueta 'Offence'
actions_to_remove = offence_actions[:1000]

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

sorted_labels_train = [item for item in sorted_labels_train if item not in actions_to_remove_list]

In [25]:
for action in actions_to_remove_list:
    dic_ordenado_train.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_train.keys())

Diccionario después de eliminar las acciones: dict_keys(['action_19', 'action_25', 'action_32', 'action_33', 'action_38', 'action_40', 'action_55', 'action_58', 'action_63', 'action_85', 'action_98', 'action_121', 'action_132', 'action_151', 'action_162', 'action_176', 'action_193', 'action_203', 'action_214', 'action_220', 'action_223', 'action_224', 'action_228', 'action_231', 'action_237', 'action_238', 'action_241', 'action_242', 'action_243', 'action_246', 'action_252', 'action_253', 'action_254', 'action_259', 'action_269', 'action_277', 'action_301', 'action_325', 'action_328', 'action_331', 'action_345', 'action_349', 'action_352', 'action_359', 'action_369', 'action_371', 'action_373', 'action_380', 'action_390', 'action_394', 'action_407', 'action_424', 'action_443', 'action_445', 'action_449', 'action_456', 'action_461', 'action_473', 'action_476', 'action_477', 'action_484', 'action_486', 'action_491', 'action_497', 'action_501', 'action_505', 'action_506', 'action_507', 'a

In [26]:
X_train  =np.array(list(dic_ordenado_train.values()))
labels_dic_train = dict(sorted_labels_train)
Y_train = np.array(list(labels_dic_train.values()))

X_test = np.array(list(dic_ordenado_test.values()))
labels_dict_test = dict(sorted_labels_test)
Y_test = np.array(list(labels_dict_test.values()))

print("Ejemplo de etiquetas en Y_train:", Y_train[:10])
print("Ejemplo de etiquetas en Y_valid:", Y_test[:10])

label_encoder = LabelEncoder()

# Ajustamos el codificador con TODAS las etiquetas posibles
label_encoder.fit(np.concatenate([Y_train, Y_test]))

# Transformamos las etiquetas de texto a números
Y_train_encoded = label_encoder.transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

print("Etiquetas originales:", np.unique(Y_train)) 
print("Etiquetas codificadas:", np.unique(Y_train_encoded))
print("Diccionario de conversión:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Ejemplo de etiquetas en Y_train: ['No offence' 'No offence' 'No offence' 'No offence' 'No offence'
 'No offence' 'No offence' 'No offence' 'No offence' 'No offence']
Ejemplo de etiquetas en Y_valid: ['Offence' 'Offence' 'Offence' 'Offence' 'Offence' 'Offence' 'No offence'
 'Offence' 'Offence' 'No offence']
Etiquetas originales: ['No offence' 'Offence']
Etiquetas codificadas: [0 1]
Diccionario de conversión: {np.str_('No offence'): np.int64(0), np.str_('Offence'): np.int64(1)}


In [27]:
print(f"X_train type: {type(X_train)}, Y_train type: {type(Y_train)}")
print(f"X_valid type: {type(X_test)}, Y_valid type: {type(Y_test)}")

# Verificar las dimensiones
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_valid shape: {X_test.shape}, Y_valid shape: {Y_test.shape}")




X_train type: <class 'numpy.ndarray'>, Y_train type: <class 'numpy.ndarray'>
X_valid type: <class 'numpy.ndarray'>, Y_valid type: <class 'numpy.ndarray'>
X_train shape: (2943, 1, 400), Y_train shape: (2943,)
X_valid shape: (301, 1, 400), Y_valid shape: (301,)


In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)

print("X_train shape después de aplanar:", X_train_flattened.shape)
print("X_valid shape después de aplanar:", X_test_flattened.shape)

# # Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train_encoded)



# Crear el modelo
rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train_resampled, Y_train_resampled)

# Predecir en el conjunto de validación
Y_test_pred = rf_model.predict(X_test_flattened)

# Evaluar el rendimiento
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Random Forest Accuracy: {accuracy}")


X_train shape después de aplanar: (2916, 400)
X_valid shape después de aplanar: (301, 400)
Random Forest Accuracy: 0.3089700996677741


In [28]:

# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)

In [29]:
print(np.unique(Y_train, return_counts=True))

(array(['No offence', 'Offence'], dtype='<U10'), array([1448, 1495]))


In [112]:
  # Crear y entrenar el modelo XGBoost
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_resampled, Y_train_resampled)

# Evaluar el modelo
y_pred = model.predict(X_test_flattened)
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo: {accuracy:.4f}")

F:\MVFoulRecognition\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:51:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Precisión del modelo: 0.3090


In [30]:
from sklearn.model_selection import train_test_split

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)


from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Definir el espacio de búsqueda
param_dist = {
    'C': uniform(0.1, 10),  # C entre 0.1 y 10
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],  # Solo para kernel 'poly'
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

# Definir el modelo base
svc = SVC()

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Predecir en test
best_svm = random_search.best_estimator_
Y_test_pred = best_svm.predict(X_test)

# Evaluar
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))



Fitting 3 folds for each of 30 candidates, totalling 90 fits
Mejores parámetros encontrados: {'C': np.float64(1.9182496720710063), 'class_weight': 'balanced', 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}
Precisión del modelo optimizado: 0.8197226502311248
Matriz de Confusión:
[[200  97]
 [ 20 332]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.91      0.67      0.77       297
           1       0.77      0.94      0.85       352

    accuracy                           0.82       649
   macro avg       0.84      0.81      0.81       649
weighted avg       0.84      0.82      0.82       649



In [34]:
with open("modelo_xgb_slowfast.pkl", "wb") as f:
    pickle.dump(best_xgb, f)

In [33]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Número de árboles
    'learning_rate': uniform(0.01, 0.2),         # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7],                # Profundidad máxima de los árboles
    'min_child_weight': [1, 2, 3, 4],            # Peso mínimo de los hijos
    'subsample': uniform(0.6, 0.4),              # Submuestra de las instancias
    'colsample_bytree': uniform(0.6, 0.4),       # Submuestra de las columnas
    'gamma': [0, 0.1, 0.2, 0.3],                 # Penalización por complejidad
    'class_weight': ['balanced', None],          # Manejo del desbalance
}

# Definir el modelo XGBoost
xgb = XGBClassifier(random_state=42)

# Realizar Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',  # Usar F1 macro como métrica de optimización
    cv=3,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Para usar todos los núcleos del procesador
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados de los mejores parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Usar el mejor modelo
best_xgb = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
Y_test_pred = best_xgb.predict(X_test)

# Evaluar el modelo
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits


F:\MVFoulRecognition\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [18:43:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros encontrados: {'class_weight': 'balanced', 'colsample_bytree': np.float64(0.7219125032632117), 'gamma': 0.1, 'learning_rate': np.float64(0.11214946051551315), 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 500, 'subsample': np.float64(0.8769744131561081)}
Precisión del modelo optimizado: 0.8073959938366718
Matriz de Confusión:
[[205  92]
 [ 33 319]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.86      0.69      0.77       297
           1       0.78      0.91      0.84       352

    accuracy                           0.81       649
   macro avg       0.82      0.80      0.80       649
weighted avg       0.82      0.81      0.80       649

